In [2]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

train_dir = './train_images'
test_dir = './test_images'

transform = transforms.Compose(
    [transforms.Grayscale(),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0,),std=(1,))])

train_data = torchvision.datasets.ImageFolder(train_dir, transform=transform)
test_data = torchvision.datasets.ImageFolder(test_dir, transform=transform)

train_data_size = len(train_data)
test_data_size = len(test_data)

In [ ]:
from torch.utils.data.sampler import SubsetRandomSampler

valid_size = 0.2
batch_size = 32
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

num_train = len(train_data)
indices_train = list(range(num_train))
np.random.shuffle(indices_train)
split_tv = int(np.floor(valid_size * num_train))
train_new_idx, valid_idx = indices_train[split_tv:],indices_train[:split_tv]

train_sampler = SubsetRandomSampler(train_new_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=5)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, num_workers=5)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=5)
classes = ('noface','face')

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 6 * 6, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model_3_layers_no_augmentation = Net().to(device)

In [ ]:
def train_NN(n_epochs, optimizer, model, criterion):
    for epoch in range(1, n_epochs+1):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print(f'[{epoch}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0

    print('Finished Training')

In [4]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer_SGD = optim.SGD(model_3_layers_no_augmentation.parameters(), lr=0.001, momentum=0.9)
n_epochs = 20

train_NN(n_epochs, optimizer_SGD, model_3_layers_no_augmentation, criterion)

# save the model after training
PATH = './1_no_augmentation.pth'
torch.save(model_3_layers_no_augmentation.state_dict(), PATH)

[2,  2000] loss: 0.578
[3,  2000] loss: 0.159
[4,  2000] loss: 0.086
[5,  2000] loss: 0.059
[6,  2000] loss: 0.043
[7,  2000] loss: 0.034
[8,  2000] loss: 0.030
[9,  2000] loss: 0.025
[10,  2000] loss: 0.023
[11,  2000] loss: 0.019
[12,  2000] loss: 0.017
[13,  2000] loss: 0.015
[14,  2000] loss: 0.013
[15,  2000] loss: 0.012
[16,  2000] loss: 0.011
[17,  2000] loss: 0.010
[18,  2000] loss: 0.010
[19,  2000] loss: 0.009
[20,  2000] loss: 0.008
[21,  2000] loss: 0.007
Finished Training


In [ ]:
def get_classifier_accuracy(model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

In [7]:
model_accuracy = get_classifier_accuracy(model_3_layers_no_augmentation)
print('Accuracy of the 3-layer no augmentation classifier on the {%d} test images: {:.2f}'.format(test_data_size, model_accuracy))

Accuracy of the network on the 10000 test images: 92 %
